# *atommovr* tutorial: the basics

this notebook walks users through the basic functionality of the `atommovr` simulation framework.

the `atommovr` framework contains two main arms: `.utils` (imported as 'movr') and `.algorithms` (imported as 'algos'):
- 'movr' contains:
  - under-the-hood code
    - objects for keeping track of moves and the state of our atom array
  - various utility functions
  - animation code generating snapshots of the array and gifs
- 'algos' contains:
  - implementations of algorithms in the literature
    - single-species
      - hungarian
      - parallel hungarian (developed in this work)
      - balance and compact (Vala 2004, reimplemented for tweezers in this work)
    - dual-species
      - inside out (developed in this work)
      - parallel hungarian (developed in this work)
  - functions to extract numerical lower bounds for rearrangement

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import atommover
import atommover.utils as movr
import atommover.algorithms as algos

# NB: the first time you run this box, it may take a little while.

### Basic Functionality

In *atommovr*, a tweezer array is represented by an instance of the class `AtomArray`.

We can always look at the state of the array by calling the `.image()` function on an instance of the class.

In [ ]:
array = movr.AtomArray([20,20], n_species=1)
array.image()

Of course, we haven't filled it yet, so there's nothing there. Let's add some atoms! 

We can simulate the process of stochastic loading through calling the `.load_tweezers()` function, and see what we get with `.image()` again.

In [ ]:
array.load_tweezers()
array.image()

That's better! Now that we have atoms, we can rearrange them into some target configuration. But first, let's figure out how to do a simple move. 

In `atommovr`, moves are represented by `Move` classes, which take four arguments: the row and column indices of the atom you want to move, and the row and column indices of the site you want to move it to. To keep benchmarking and error modelling simple and intuitive, we only move an atom over by a single row and/or column in a single move, and break longer moves up into a series of `Move` objects.

To apply a move to the array, we can call the `.move_atoms(move_list)` function, and it will apply the moves to the atom.

To see a visualization of our move, we can call `.image(move_list)`.

In [ ]:
atom_row, atom_col = 4, 3     # fill these in yourself! #
target_row, target_col = 4, 4

move = movr.Move(atom_row, atom_col, target_row, target_col)

move_list = [move]

array.move_atoms(move_list)

array.image(move_list)


Great! Now, let's say we want to be efficient and move multiple atoms simultaneously. We can simply do this by calling two instances of `Move` objects and adding both to our `move_list`.

In [ ]:
atom_row, atom_col = 7, 5 # fill these in yourself!
target_row, target_col = 8, 5

move1 = movr.Move(atom_row, atom_col, target_row, target_col)

atom_row2, atom_col2 = 7, 6 # fill these in yourself!
target_row2, target_col2 = 8, 6

move2 = movr.Move(atom_row2, atom_col2, target_row2, target_col2)

move_list = [move1, move2]

array.move_atoms(move_list)

array.image(move_list)

Great! However, it was already a lot of work to move that one atom, and we don't want to be stuck doing this manually every time.

Instead, what we can do is call one of the built-in rearrangement algorithms. Each algorithm is also represented by its own class, like `Hungarian`.

To tell it what to prepare, we first have to call our array's `.generate_target()` function (which by default, generates a nice checkerboard pattern). There are prebuilt target configurations to generate in the `Configurations` class in `atommover.utils`, or you can specify your own manually by setting the `.target` variable of your array.

Now that we have a target configuration in mind, we can have the algorithm compute a list of moves to prepare it by calling its `.get_moves()` function, and passing our array as an argument. 

In [ ]:
algo = algos.Hungarian()
array.generate_target(movr.Configurations.MIDDLE_FILL, occupation_prob = 0.6)
final_config, move_set, success_flag = algo.get_moves(array)
print(move_set)

Nice! Now, we have prepared our checkerboard pattern, but we notice that there might be some excess atoms on the grid. 

To get rid of these, we can call the algorithm's `.get_moves()` function with the kwarg `do_ejection = True`.

This adds an ejection subroutine where excess atoms are removed after all the target sites are filled. You can run it on other algorithms too, or easily add it to your own.

In [ ]:
array.matrix = array.last_loaded_config # resetting the state of the array to the initial loading state
final_config, move_set, success_flag = algo.get_moves(array, do_ejection = True)

move_time, _ = array.evaluate_moves(move_set)

array.image()

 tyhuijoAs an alternative to calling `.evaluate_moves()`, we can visualize the whole process in a gif/movie, with the `make_single_species_gif()` function from `atommover.utils`.

Look in your `\figs` folder to see the gif!

In [ ]:
array.matrix = array.last_loaded_config # resetting the state of the array to the initial loading state

move_time = movr.make_single_species_gif(array, move_set, savename='first_gif')

Two notes: 

- 'ejection moves' (where an atom is moved off of the grid) are marked by green dots instead of arrows.
- although we rearranged the atoms successfully, the Hungarian algorithm is slow because it only moves one atom at a time. Other algorithms, like `algos.BCv2()`, move atoms in parallel, which enables faster assembly, at the cost of more tweezer moves.

  - For more insight into the tradeoff between these algorithms, see Fig. 4 in the paper.
  - Curious about making your own algorithm? See the next section and `algos.Algorithm.py`.

### Basic Algorithm Implementation: finding the optimal 1D rearrangement algorithm

Now that we've warmed up a bit, let's level up: now, we'll find an optimal solution for the rearrangement problem in 1D together.

To figure this problem out, we'll first start small with a chain of 9 atoms. Suppose we are given the following initial and target configurations:

In [ ]:
import numpy as np

n_atoms = 9
array = movr.AtomArray([1,n_atoms])
array.target = np.array([0,0,1,0,1,0,1,0,0]).reshape([1,n_atoms,1]) # given target configuration
array.matrix = np.array([0,1,0,0,0,0,1,1,0]).reshape(np.shape(array.target)) # given initial configuration

print('Given Target Configuration:')
array.plot_target_config()

print('Given Initial Configuration:')
array.image()

In the special case where we have exactly the same number of atoms as target sites, the optimal algorithm is very simple: 

- Starting from the left (right) side, we can pair the leftmost (rightmost) atom with the leftmost (rightmost) target site, and move it one space in that direction. 
- We can then iterate through all the atoms and do the same thing.

And the best thing about 1d is: we can do multiple moves simultaneously!

In [ ]:
import copy

def special_case_algo_1d(init_config: np.ndarray, target_config: np.ndarray):
    arr_copy = movr.AtomArray(np.shape(init_config)[:2])
    arr_copy.target = copy.deepcopy(target_config)
    arr_copy.matrix = copy.deepcopy(init_config)
    
    # first, find the column indices of the target sites
    # and those of the sites with atoms
    target_indices = np.where(arr_copy.target == 1)[1]
    atom_indices = np.where(arr_copy.matrix == 1)[1]
    
    if len(target_indices) != len(atom_indices):
        raise Exception(f"Number of atoms ({len(atom_indices)}) does not equal number of target sites ({len(target_indices)}).")

    # second, we can pair the atoms and make a list
    pairs = []
    for ind, target_index in enumerate(target_indices):
        atom_index = atom_indices[ind]
        pair = (target_index, atom_index)
        pairs.append(pair)
    # lastly, we can move atoms towards their target positions
    target_prepared = np.array_equal(arr_copy.target, arr_copy.matrix)
    move_set = []
    while not target_prepared:
        move_list = []
        for i, pair in enumerate(pairs):
            target_index, atom_index = pair
            if target_index != atom_index:
                new_atom_index = int(atom_index+np.sign(target_index-atom_index))
                move = movr.Move(0, atom_index, 0, new_atom_index)
                move_list.append(move)
                pairs[i] = (target_index, new_atom_index)
        if move_list != []:
            _, _ = arr_copy.evaluate_moves([move_list])
            move_set.append(move_list)
        else:
            break
        # target_prepared = np.array_equal(arr_copy.target, arr_copy.matrix)
    
    return move_set, atom_indices

move_set, atom_indices = special_case_algo_1d(array.matrix, array.target)
move_time = movr.make_single_species_gif(array, move_set, savename='first_algo')

Success!

We'll find that in the general case (where $n_{\text{atoms}} > n_{\text{targets}}$), the problem of finding the optimal pairing between atoms and target sites is a little more difficult. 

To make this a little easier on ourselves and build some intuition, let's consider the simplest case of a fully-filled target configuration:

In [ ]:
array.matrix = np.array([0,1,0,1,0,0,1,0,1]).reshape(np.shape(array.target))
n_targets = 3
array.generate_target(movr.Configurations.MIDDLE_FILL, middle_size=[1,n_targets])

print('Target Configuration:')
array.plot_target_config()

print('Initial Loading Configuration:')
array.image()

We can think of decomposing this 4-atom, 3-target problem into a 3-atom, 3-target problem (which we know how to solve above). 

But that raises the important question: which atom (of the 4) do we choose to ignore? It is clear that we should keep the innermost two atoms, as they are closest to the target sites in question. That leaves the two outermost atoms:

In [ ]:
# ignoring the leftmost atom
array.matrix[0,1,0] = 0
move_set, _ = special_case_algo_1d(array.matrix, array.target)
array.matrix[0,1,0] = 1
print(f'Number of moves (ignoring leftmost atom): {len(move_set)}')

# ignoring the rightmost atom
array.matrix[0,-1,0] = 0
move_set1, _ = special_case_algo_1d(array.matrix, array.target)
array.matrix[0,-1,0] = 1
print(f'Number of moves (ignoring rightmost atom): {len(move_set1)}')

We see that ignoring the rightmost atom is the optimal choice in this situation.

A very pragmatic strategy, then, would be to select the centermost set of atoms $C$, and calculate the farthest distance that an atom in set $C$ would have to move to make the target configuration. Then, we could look at the sets to the left and right of these ($L$ and $R$), and if the farthest distance is shorter than in $C$, we iterate this procedure until we find the set with the lowest number of atoms.

In [ ]:
# utility function that calculates the longest move distance between target sites and atom sites
def find_largest_dist_to_move(target_inds, atom_inds):
    if len(target_inds) > len(atom_inds):
        return np.inf
    max_dist = 0
    for ind, target_loc in enumerate(target_inds):
        atom_loc = atom_inds[ind]
        distance = np.abs(target_loc-atom_loc)
        if distance > max_dist:
            max_dist = distance
    return max_dist

def middle_fill_algo_1d(init_config: np.ndarray, target_config: np.ndarray):
    arr_copy = movr.AtomArray(np.shape(init_config)[:2])
    arr_copy.target = copy.deepcopy(target_config)
    arr_copy.matrix = copy.deepcopy(init_config)
    # first, find the column indices of the target sites
    # and those of the sites with atoms
    target_indices = np.where(arr_copy.target == 1)[1]
    atom_indices = np.where(arr_copy.matrix == 1)[1]
    n_targets = len(target_indices)

    # second, find the optimal pairing of atoms if 
    if n_targets == len(atom_indices):
        return special_case_algo_1d(init_config, target_config)
    elif n_targets > len(atom_indices):
        return [], []
    
    # third, find the centermost set of atoms
    avg_targ_pos = int(np.ceil(np.mean(target_indices)))
    count = 0
    sufficient_atoms = False
    while not sufficient_atoms:
        center_region = arr_copy.matrix[0,avg_targ_pos-count:avg_targ_pos+count+1]
        n_atoms_in_center_region = np.sum(center_region)
        sufficient_atoms = n_targets <= n_atoms_in_center_region
        if not sufficient_atoms:
            count+=1
        else:
            break
    first_atom_loc = np.where(center_region == 1)[0][0]+avg_targ_pos-count

    # fourth, look to the adjacent sets and see if these are better
    look_right = True
    right_count = 0
    while look_right:
        list_ind = np.where(atom_indices==first_atom_loc)[0][0]
        current_r_atom_set = atom_indices[list_ind+right_count:list_ind+right_count+n_targets]
        right_atom_set = atom_indices[list_ind+right_count+1:list_ind+right_count+n_targets+1]
        dist_r_current = find_largest_dist_to_move(target_indices, current_r_atom_set)
        dist_right = find_largest_dist_to_move(target_indices, right_atom_set)
        if dist_right > dist_r_current:
            look_right = False
        else:
            right_count += 1
    
    look_left = True
    left_count = 0
    while look_left:
        list_ind = np.where(atom_indices==first_atom_loc)[0][0]
        current_l_atom_set = atom_indices[list_ind-left_count:list_ind-left_count+n_targets]
        left_atom_set = atom_indices[list_ind-left_count-1:list_ind-left_count+n_targets-1]
        dist_l_current = find_largest_dist_to_move(target_indices, current_l_atom_set)
        dist_left = find_largest_dist_to_move(target_indices, left_atom_set)
        if dist_left > dist_l_current:
            look_left = False
        else:
            left_count += 1
    
    if dist_l_current < dist_r_current:
        best_atom_set = current_l_atom_set
    else:
        best_atom_set = current_r_atom_set
    
    # fifth, find the best set and assign pairs
    pairs = []
    for ind, target_index in enumerate(target_indices):
        atom_index = best_atom_set[ind]
        pair = (target_index, atom_index)
        pairs.append(pair)

    # lastly, we can move atoms towards their target positions
    target_prepared = np.array_equal(arr_copy.target, arr_copy.matrix)
    move_set = []
    while not target_prepared:
        move_list = []
        for i, pair in enumerate(pairs):
            target_index, atom_index = pair
            if target_index != atom_index:
                new_atom_index = int(atom_index+np.sign(target_index-atom_index))
                move = movr.Move(0, atom_index, 0, new_atom_index)
                move_list.append(move)
                pairs[i] = (target_index, new_atom_index)
        if move_list != []:
            _, _ = arr_copy.evaluate_moves([move_list])
            move_set.append(move_list)
        else:
            target_prepared = True
    
    return move_set, best_atom_set

In [ ]:
array = movr.AtomArray([1,21], params=movr.PhysicalParams(loading_prob=0.5))

print('Target Configuration:')
array.generate_target(movr.Configurations.MIDDLE_FILL, middle_size = [1,9])
print('Initial Configuration:')
array.load_tweezers()
array.plot_target_config()
if np.sum(array.matrix) > np.sum(array.target):
    array.image()
    move_set, _ = middle_fill_algo_1d(array.matrix, array.target)
    _, _ = array.evaluate_moves(move_set)
    print('Final Configuration:')
    array.image(move_set[-1])

### Basic analysis implementation: using `ErrorModel` and `Benchmarking`

`atommovr` contains different objects for simulating and comparing the performance of different algorithms in experimental settings. Two of these objects are the `Benchmarking` class and the `ErrorModel` class (and its various child classes). 

We'll look at these separately below:

1. `ErrorModel` and child classes

    In `atommovr`, error models are objects that simulate specific loss processes, with parameters that can be set by the user. To apply an error model, you can pass it as an argument when you make an `AtomArray` object, or you can assign it later by setting the `AtomArray.error_model` class attribute.

    Here's an example of assigning the simplest error model, `ZeroNoise`. As the name suggests, this error model simulates perfect rearrangement, where all moves succeed with 100% probability and there are no additional loss mechanisms. (Although you didn't know, you actually have already used this error model in the above sections - it is the default assignment for the `AtomArray.error_model` attribute).

In [ ]:
# Two ways of using an error model
noiseless_model = movr.ZeroNoise()
arr = movr.AtomArray(error_model=noiseless_model)

arr1 = movr.AtomArray()
arr1.error_model = noiseless_model

This is a bit boring, so let's use another error model, `UniformVacuumTweezerError`:

As the name suggests, this model simulates two loss processes: atom loss from collisions with background molecules (which arise due to imperfect vacuum) and tweezer failure (to either pick up an atom, or put down an atom). If a tweezer pickup error occurs for a move, the atom remains in the trap (i.e. the move is effectively not performed). If a tweezer putdown error occurs for a move, the atom is lost from the array.

It has five tunable parameters, which can be specified either when the model is called or by modifying the corresponding class attribute later:

*Atom loss from background collisions*
- `UniformVacuumTweezerError.lifetime` (s) - the lifetime of an individual atom in a trap.
- `UniformVacuumTweezerError.pickup_time` (s) - the time it takes to transfer an atom from the static tweezer to the moving tweezer.
- `UniformVacuumTweezerError.putdown_time` (s) - the time it takes to transfer an atom from the moving tweezer to the static tweezer.

*Tweezer loss*
- `UniformVacuumTweezerError.pickup_fail_rate` (%, in decimal form) - the percentage of moves that fail to pick up the atom.
- `UniformVacuumTweezerError.putdown_fail_rate` (%, in decimal form) - the percentage of moves that fail to put down the atom.

Let's get to it, and look at the default values for these parameters (which are chosen to roughly mimic a typical atom array experiment). You can try setting these yourself as well.

In [ ]:
noisy_model = movr.UniformVacuumTweezerError()
print(f'Lifetime: {noisy_model.lifetime} s')
print(f'Tweezer pickup time: {noisy_model.pickup_time} s')
print(f'Tweezer putdown time: {noisy_model.putdown_time} s')
print(f'Tweezer pickup failure rate: {noisy_model.pickup_fail_rate} ({noisy_model.pickup_fail_rate*100}%)')
print(f'Tweezer putdown failure rate: {noisy_model.putdown_fail_rate} ({noisy_model.putdown_fail_rate*100}%)')

Now, let's see how things are different in a noisy environment (we'll increase the putdown failure rate to 10% for illustrative purposes, but try playing around with the different parameters yourself!)

In [ ]:
# modifying parameters
noisy_model.putdown_fail_rate = 0.1 # 10%

# calling array and algo objects
array = movr.AtomArray(error_model=noisy_model)
algo = algos.Hungarian()

# preparing our target configuration and calculating the move set, as usual
array.load_tweezers()
array.generate_target()
config, move_set, success_flag = algo.get_moves(array, do_ejection = True)

# making a gif of noisy rearrangement
move_time = movr.make_single_species_gif(array, move_set)

From looking at the gif, you might see that the algorithm does not always succeed in preparing the target configuration, and that some moves fail. Move failures are marked by different colors: if an atom is not picked up, it is marked in yellow, whereas an atom that is not put down and lost is marked in magenta. 

(Note: you can customize these colors, and also the color of the atoms and tweezer spots, in `movr.utils.customize.py`!)

2. `Benchmarking`
   - Now that we know how to use error models, we can study the performance of an algorithm along a spectrum of noise settings. The object in `movr.Benchmarking`  is designed to make such sweeps convenient and straightforward.
   - This object supports the comparison of multiple algorithms across sweeps of target configurations, error models, physical properties of the tweezer array (such as spacing), system sizes, and rounds of rearrangement, with support for both single- and dual-species algorithms.
   - Let's try initializing a benchmarking object with some error models, and vary the tweezer error:

In [ ]:
# setting parameters
algorithms = [algos.Hungarian()]
n_shots = 100
sys_sizes = [10] # 10 x 10 array

# preparing list of error models with different tweezer error probabilities
tweezer_pickup_errors = [0.01, 0.05, 0.1, 0.15]
error_models = []
for pickup_error in tweezer_pickup_errors:
    mod = movr.UniformVacuumTweezerError(pickup_fail_rate=pickup_error)
    error_models.append(mod)

# initializing benchmarking object
bench = movr.Benchmarking(algorithms, sys_sizes = sys_sizes, error_models_list=error_models)

Great! Now, all that's left is to run it, which we can do with the `.run()` function. We can see the results, which are stored nicely in an `xarray` dataset, by looking at the `.benchmarking_results` attribute.

In [ ]:
bench.run()
bench.benchmarking_results

The dataset shows both the metadata containing the sweep parameters (see the 'Coordinates' and 'Indexes' tabs) and useful output data (success rate, rearrangement time, filling fraction of the target sites, the number of atoms in incorrect spots, the number of atoms after rearrangement, and the number of target sites).

For this dataset, we can look at the success rate as a function of the tweezer error.

In [ ]:
# retrieving success rates from xarray object
success_arr = bench.benchmarking_results['success rate']
rates = success_arr.to_numpy()[0,0,0,:,0,0]

# plotting code
plt.scatter(tweezer_pickup_errors, rates)
plt.errorbar(tweezer_pickup_errors, rates, rates/np.sqrt(n_shots)) # shot noise errorbars
plt.xlabel('Tweezer error')
plt.ylabel('Success rate')

And with that, you're officially an atom mover!

If you're interested in modifying the existing error model, or making a more specific model to fit your exact experimental setting, please see `movr.utils.ErrorModel.py` for a guide.